# Final Project: iMet Classification Task
## CSC 594: Advanced Deep Learning
## Ross Pollock

In [ ]:
! pip install tensorflow_addons -U

     |████████████████████████████████| 1.1MB 23.0MB/s 
  Found existing installation: tensorflow-addons 0.8.3
    Uninstalling tensorflow-addons-0.8.3:
      Successfully uninstalled tensorflow-addons-0.8.3


In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as L
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score

from google.colab import drive 
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
! tar zxf /drive/MyDrive/CSC594/Data/Image/imet.tar.gz -C /content/

In [ ]:
def to_int_labels(str_labels,delimiter=' '):
  '''
  Transform string of integers
  seperated by delimiter to 
  list of integers

  Args:
    str_labels: str
    delimiter: char
  '''
  x = str_labels.split(delimiter)
  x = list(map(lambda x: int(x.strip()),x))
  return x 


@tf.function
def smart_resize(img,size=244):
  h,w = tf.shape(img)[0], tf.shape(img)[1]
  min_dim = tf.minimum(h,w)
  ratio = size / tf.cast(min_dim,tf.float32) 

  new_w = tf.cast(w,tf.float32) * ratio
  new_h = tf.cast(h,tf.float32) * ratio

  img = tf.image.resize(img,[new_h,new_w],preserve_aspect_ratio=True)
  img = tf.image.resize_with_crop_or_pad(img,size,size)
  return img



@tf.function
def img_preprocess(fpath):
  '''
  Load and Preprocess JPEG IMG 
  from file path

  Args:
    fpath: str
  Returns:
    tf.tensor 
      - shape (244,244,3)
  '''
  img = tf.io.read_file(fpath)
  img = tf.image.decode_jpeg(img,3)
  img = tf.cast(img,tf.float32)

  img = smart_resize(img,244)
  #img = tf.image.resize_with_crop_or_pad(img,244,244)

  img /= (255/2)
  img -= 1 
  return img 

## ResNet Implementation

In [ ]:
class ResNetBlock(L.Layer):
  def __init__(self,n_filters,regularizer=keras.regularizers.L2(1e-3),activation=L.ReLU,start=False,downsample=False):
    super(ResNetBlock,self).__init__()
    self.regularizer = regularizer
    self.activation = activation
    self.n_filters = n_filters
    self.start = start
    self.stride = 2 if downsample else 1


    self.left_path = keras.Sequential(
        [
          L.Conv2D(self.n_filters,1,self.stride,padding='same',kernel_regularizer=self.regularizer),
          L.BatchNormalization(),
          self.activation(),
          L.Conv2D(self.n_filters,3,1,padding='same',kernel_regularizer=self.regularizer),
          L.BatchNormalization(),
          self.activation(),
          L.Conv2D(self.n_filters*4,1,1,padding='same',kernel_regularizer=self.regularizer),
          L.BatchNormalization()
        ]
    )
  

    
    if self.start:
      self.right_path = keras.Sequential(
          [
           L.Conv2D(self.n_filters*4,1,self.stride,'same',kernel_regularizer=self.regularizer),
           L.BatchNormalization()
          ]
      )

    

  def call(self,input_tensor,training=False):
    x = self.left_path(input_tensor)
    if self.start:
      y = self.right_path(input_tensor)
    else:
      y = input_tensor

    z = tf.add(x,y)
    return self.activation()(z)
      


class ResNetStack(L.Layer):
  def __init__(self,n_blocks,n_filters,first_block=False,**kwargs):
    super(ResNetStack,self).__init__()
    blocks = [ ResNetBlock(n_filters,start=(not i),downsample=(not i and not first_block),**kwargs) for i in range(n_blocks)]
    self.stack = keras.Sequential(
        blocks
    )
    self.out_dim = n_filters * 4

  def call(self,input_tensor,training=False):

    return self.stack(input_tensor) 

def ResNet(n_layers,width=1,input_shape=None,**kwargs):
    if n_layers not in [50,101,152]:
      raise ValueError
    
    model = keras.Sequential([
        L.Input(input_shape),
        L.Conv2D(64,7,2,padding='same'),
        L.BatchNormalization(),
        L.ReLU(),
        L.MaxPool2D(3,2,'same')
        
    ],name=f'ResNet{n_layers}')

    model_specs = {
        50: [3,4,6,3],
        101: [3,4,23,3],
        152: [3,8,36,3]
    }
    filters = list(map(lambda x: x*width,[64,128,256,512]))

    for i,(stack,f) in enumerate(zip(model_specs[n_layers],filters)):
      if i == 0:
        stack = ResNetStack(stack,f,first_block=True)
      else:
        stack = ResNetStack(stack,f,first_block=False)
      model.add(stack)
    model.add(L.GlobalAveragePooling2D())
    return model

## Load Datasets paths and labels

In [ ]:
# Read Metadata for iMet dataset
df_train = pd.read_csv('/drive/My Drive/CSC594/Data/imet_train_split.csv')

# Transform image ids to absolute file path of image
train_paths = '/content/imet_reduced/train/' + df_train.id + '.jpg'

# Extract labels and transform from strings to list of ints
y_train = df_train.attribute_ids.map(to_int_labels).to_list()

# Transform to Binary Matrix
label_encoder = MultiLabelBinarizer()
y_train = label_encoder.fit_transform(y_train)
y_train = y_train.astype(np.float32)
n_train_samples, n_classes = y_train.shape

In [ ]:
# Read Metadata for iMet dataset
df_test = pd.read_csv('/drive/My Drive/CSC594/Data/imet_test_split.csv')

# Transform image ids to absolute file path of image
test_paths = '/content/imet_reduced/train/' + df_test.id + '.jpg'

# Extract labels and transform from strings to list of ints
y_test = df_test.attribute_ids.map(to_int_labels).to_list()
y_test = label_encoder.transform(y_test)
n_test_samples = y_test.shape[0]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) [328] will be ignored
  .format(sorted(unknown, key=str)))


## Load Pre-trained Model

In [ ]:

# Download and extract model
model_url = 'https://rosspollock.design/models/resnet50120.tar.gz'
keras.utils.get_file('modelx',model_url,untar=True,extract=True)
encoder = keras.models.load_model('/root/.keras/datasets/query_encoder120k')
cnn_name = encoder.layers[0].name
encoder = encoder.get_layer(cnn_name)
if encoder.name == 'ResNet50':
  new_encoder = ResNet(50,2,input_shape=(244,244,3))
  new_encoder.set_weights(encoder.get_weights())
  encoder = new_encoder 
  del new_encoder
_, rep_dim = encoder.output_shape

In [ ]:
def prep_ds(file_paths,preprocesser=img_preprocess,batch_size=128):
  ds = tf.data.Dataset.from_tensor_slices(file_paths)
  ds = ds.map(preprocesser,tf.data.experimental.AUTOTUNE)
  ds = ds.batch(batch_size)
  return ds

def prep_with_labels(file_paths,labels,preprocesser=img_preprocess,batch_size=128,train=False,repeat=False):
  ds_img = tf.data.Dataset.from_tensor_slices(file_paths)
  ds_img = ds_img.map(preprocesser,tf.data.experimental.AUTOTUNE)
  ds_labels = tf.data.Dataset.from_tensor_slices(labels)
  ds = tf.data.Dataset.zip((ds_img,ds_labels))
  if train:
    ds = ds.shuffle(1000)
  ds = ds.batch(batch_size)
  if repeat:
    ds = ds.repeat()
  return ds

## Extract Representations 

In [ ]:
ds_train = prep_ds(train_paths)
ds_test = prep_ds(test_paths)

In [ ]:
x_train = np.zeros((n_train_samples,rep_dim))
x_test = np.zeros((n_test_samples,rep_dim))

In [ ]:
BATCH_SIZE=128
pbar = keras.utils.Progbar(n_train_samples // BATCH_SIZE + 1 )
for i, batch in enumerate(ds_train):
  reps = encoder(batch,training=False)
  x_train[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = reps.numpy()
  pbar.update(i+1)

pbar = keras.utils.Progbar(n_test_samples // BATCH_SIZE + 1 )
for i, batch in enumerate(ds_test):
  reps = encoder(batch,training=False)
  x_test[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = reps.numpy()
  pbar.update(i+1)

171/171 [==============================] - 230s 1s/step


## Fit Classifier

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:


multilabel_clf = keras.Sequential(
    [
    keras.layers.Input(shape=(rep_dim,)),
    keras.layers.Dense(n_classes,'sigmoid')
    ]
)

multilabel_clf.compile(
    keras.optimizers.Adam(1e-3),
    tfa.losses.SigmoidFocalCrossEntropy()
)


history = multilabel_clf.fit(x_train,y_train,epochs=50,verbose=1,validation_split=0.2)

Epoch 1/50
2185/2185 [==============================] - 8s 4ms/step - loss: 44.4770 - val_loss: 11.0358
Epoch 2/50
2185/2185 [==============================] - 7s 3ms/step - loss: 5.9124 - val_loss: 3.2160
Epoch 3/50
2185/2185 [==============================] - 7s 3ms/step - loss: 2.3304 - val_loss: 1.7917
Epoch 4/50
2185/2185 [==============================] - 7s 3ms/step - loss: 1.5699 - val_loss: 1.4423
Epoch 5/50
2185/2185 [==============================] - 7s 3ms/step - loss: 1.3669 - val_loss: 1.3500
Epoch 6/50
2185/2185 [==============================] - 7s 3ms/step - loss: 1.3042 - val_loss: 1.3262
Epoch 7/50
2185/2185 [==============================] - 7s 3ms/step - loss: 1.2797 - val_loss: 1.3215
Epoch 8/50
2185/2185 [==============================] - 7s 3ms/step - loss: 1.2664 - val_loss: 1.3169
Epoch 9/50
2185/2185 [==============================] - 7s 3ms/step - loss: 1.2572 - val_loss: 1.3163
Epoch 10/50
2185/2185 [==============================] - 7s 3ms/step - loss: 1.2

## Evaluate Test Set Metrics

In [ ]:
y_pred = multilabel_clf.predict(x_test)

In [ ]:
THRESHOLD = 0.20
for weighting in ['micro','macro','samples']:
  y_pred_t = (y_pred >= THRESHOLD).astype(np.int32)
  fscore = fbeta_score(y_test,y_pred_t,beta=2.0,average=weighting,zero_division=0)
  print(f'F2 {weighting} score: {fscore:.4f}')

F2 micro score: 0.3360
F2 macro score: 0.1011
F2 samples score: 0.3325
